# Привет!
#### Это мой проект по здравоохранению. Он состоит из 3х частей:
* __Российское здравоохранение__
* __Сравнение с международным__
* __Приложение в streamlit__

#### Для начала сделаем наш список источников и загрузим наши данные

* 	https://www.ffoms.gov.ru/documents/registry/?region=362&rgn= __(Парсинг selenium)__
*	https://rosstat.gov.ru/folder/13721 __(Напрямую скачивать)__
*	https://www.fedstat.ru __(Напрямую скачивать)__
*	https://www.newsweek.com/worlds-best-hospitals-2022 __(Парсинг)__
*	https://worldpopulationreview.com/country-rankings/best-healthcare-in-the-world
*	https://www.oecd.org __(API)__
*   https://www.worldbank.org/en/home __(API+Парсинг напрямую)__
*   https://www.healthdata.org/gbd/2019 __(Напрямую скачивать)__
*   https://vademec.ru/download/top200clinics2021.pdf __(Парсинг PDF)__
*   https://healthdata.gov
*   https://mosgorzdrav.ru/ru-RU/metricks/default/index.html?group_id=1
*   https://data.mos.ru/opendata

In [1]:
import pandas as pd
import numpy as np
import math
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import json
import re
import geopandas
from shapely.geometry import Polygon
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import datetime as dt
import pandas_datareader.data as web
import wbgapi as wb
from webdriver_manager.chrome import ChromeDriverManager

### Москва и Санкт-Петербург
Доделать Санкт-Петербург, довести все до логичного финала

In [2]:
def fs(x):
    return x.apply(lambda z: z.split(',')[0].strip())

In [3]:
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get('https://www.ffoms.gov.ru/documents/registry/?region=362&rgn=')
l=browser.find_element(By.CLASS_NAME, "btn-success")
l.click()
browser.get('https://www.ffoms.gov.ru/documents/registry/data.php')
moscow_soup = BeautifulSoup(browser.page_source)
moscow_raw=moscow_soup.body.get_text()
browser.get('https://www.ffoms.gov.ru/documents/registry/?region=362&rgn=')
select = Select(browser.find_element_by_id('selectRgnId'))
select.select_by_visible_text('г. Санкт-Петербург')
l=browser.find_element(By.CLASS_NAME, "btn-success")
l.click()
browser.get('https://www.ffoms.gov.ru/documents/registry/data.php')
spb_soup = BeautifulSoup(browser.page_source)
spb_raw=spb_soup.body.get_text()

In [4]:
def data_clean(moscow_raw):
    moscow_raw=moscow_raw[61:len(moscow_raw)-2]
    for i in re.findall('balloonContent": ".+", "c', moscow_raw):
        j=i[18:len(i)-5].replace('"',"'")
        moscow_raw=moscow_raw.replace(i[18:len(i)-5],j)
    for i in re.findall('"clusterCaption": ".+", "h', moscow_raw):
        j=i[19:len(i)-5].replace('"',"'")
        moscow_raw=moscow_raw.replace(i[19:len(i)-5],j)
    for i in re.findall('"hintContent": ".+"', moscow_raw):
        j=i[16:len(i)-2].replace('"',"'")
        moscow_raw=moscow_raw.replace(i[16:len(i)-2],j)
    moscow_raw=moscow_raw.replace('\\','')
    moscow_dataset={'hospitals':[]}
    for i in moscow_raw.split('\n'):
        if len(i.strip())>1:
            moscow_dataset['hospitals'].append(json.loads(i.strip().strip(',').strip()))
    moscow_dataset=json.loads(json.dumps(moscow_dataset))
    return moscow_dataset

In [5]:
def concat(df):
    x=[]
    for i in df['Виды медицинской помощи']:
        x.extend(str(i).split(','))
    while 'nan' in x:
        x.remove('nan')
    if 'nan' not in x:
        return list(set(x))
    else:
        print(x)
        return None

In [6]:
def make_df(moscow_dataset):
    moscow_df=[]
    for i in moscow_dataset['hospitals']:
        moscow_df.append([i['id'],i['geometry']['coordinates'][1],i['geometry']['coordinates'][0] ,i['properties']['balloonContent'],i['properties']['hintContent']])
    moscow_df=pd.DataFrame(moscow_df, columns=['id','lon','lat', 'name+adress', 'name'])
    moscow_df=gpd.GeoDataFrame(moscow_df, geometry = gpd.points_from_xy(moscow_df['lon'], moscow_df['lat']))
    moscow_df['name'] = moscow_df['name'].str.replace("'",'"')
    return moscow_df

In [7]:
spb_dataset=data_clean(spb_raw)
moscow_dataset=data_clean(moscow_raw)
moscow_df=make_df(moscow_dataset)
spb_df=make_df(spb_dataset)

In [8]:
classification=pd.read_excel('V008 Вид мед.помощи.xlsx')
classification=classification[['IDVMP', 'VMPNAME']].set_index('IDVMP').to_dict()['VMPNAME']

In [9]:
def make_class(df1,df2, classification):
    df1=df1[['Полное наименование медицинской организации', 'Виды медицинской помощи']]
    df1=df1.groupby('Полное наименование медицинской организации').apply(concat)
    df1=pd.DataFrame(df1)
    df3=df2.set_index('name').join(df1, how='left')
    df3=df1.reset_index().dropna()
    for i in df3[0]:
        for j in range(len(i)):
            i[j]=classification[int(i[j])]
    return df3

In [10]:
russia_data=pd.read_excel('МО-Портал-5.xls')
moscow_type_raw=russia_data[russia_data['Наименование субъекта Российской Федерации']=='г. Москва']
moscow_type_raw=moscow_type_raw[(fs(moscow_type_raw['Адрес (место) нахождения МО'])=='Г. МОСКВА')|(fs(moscow_type_raw['Адрес (место) нахождения МО'])=='Г.МОСКВА')]
moscow_df_with_classes=make_class(moscow_type_raw,moscow_df, classification )

In [11]:
spb_type_raw=russia_data[russia_data['Наименование субъекта Российской Федерации']=='г. Санкт-Петербург']
spb_type_raw=spb_type_raw[ (fs(spb_type_raw['Адрес (место) нахождения МО'])=='САНКТ-ПЕТЕРБУРГ')| (fs(spb_type_raw['Адрес (место) нахождения МО'])=='САНКТ-ПЕТЕРБУРГ Г')|( fs(spb_type_raw['Адрес (место) нахождения МО'])=='САНКТ-ПЕТЕРПУРГ' )| (fs(spb_type_raw['Адрес (место) нахождения МО'])=='САНКТ-ПЕТЕРБУРГ ГОРОД')]
spb_df_with_classes=make_class(spb_type_raw,spb_df, classification)
print(spb_df_with_classes)

           Полное наименование медицинской организации  \
0    АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ "МЕДИЦИН...   
1    АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ "МЕДИЦИН...   
2    АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ «МЕДИЦИН...   
3          АКЦИОНЕРНОЕ ОБЩЕСТВО "АДМИРАЛТЕЙСКИЕ ВЕРФИ"   
4              АКЦИОНЕРНОЕ ОБЩЕСТВО "БАЛТИЙСКИЙ ЗАВОД"   
..                                                 ...   
367  ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИ...   
368  ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИ...   
369  ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...   
370     ЧАСТНОЕ МЕДИЦИНСКОЕ УЧРЕЖДЕНИЕ "ЕВРОМЕДСЕРВИС"   
371  ЧАСТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИЯ "КЛИНИЧЕСКА...   

                                                     0  
0    [первичная специализированная медико-санитарна...  
1    [первичная специализированная медико-санитарна...  
2    [первичная специализированная медико-санитарна...  
3    [первичная специализированная медико-санитарна...  
4    [первичная сп

### Мир

In [12]:
datasets=['SP.POP.TOTL','NY.GDP.PCAP.CD', 'SH.XPD.CHEX.PC.CD','EN.ATM.CO2E.PC', 'NV.AGR.TOTL.ZS', 'SP.DYN.CBRT.IN', 'SP.DYN.LE00.IN' ]
data=wb.data.DataFrame(datasets, time=range(2000,2019))

In [13]:
data1=pd.melt(data.reset_index(), id_vars=['series','economy'], value_vars=list(data.columns))
data1=pd.pivot(data1,index=['economy','variable'],columns='series').reset_index().dropna()
data_for_regression=data1['value']
data_for_visualizations=data
print(data)

                                 YR2000        YR2001        YR2002  \
economy series                                                        
ABW     EN.ATM.CO2E.PC              NaN           NaN           NaN   
        NV.AGR.TOTL.ZS     4.100218e-01  4.051498e-01  4.022158e-01   
        NY.GDP.PCAP.CD     2.061775e+04  2.067037e+04  2.043689e+04   
        SH.XPD.CHEX.PC.CD           NaN           NaN           NaN   
        SP.DYN.CBRT.IN     1.417300e+01  1.376200e+01  1.337500e+01   
...                                 ...           ...           ...   
ZWE     NY.GDP.PCAP.CD     5.630575e+02  5.683863e+02  5.305304e+02   
        SH.XPD.CHEX.PC.CD           NaN           NaN           NaN   
        SP.DYN.CBRT.IN     3.177600e+01  3.196300e+01  3.227400e+01   
        SP.DYN.LE00.IN     4.464900e+01  4.401000e+01  4.352300e+01   
        SP.POP.TOTL        1.188148e+07  1.192391e+07  1.195429e+07   

                                 YR2003        YR2004        YR2005  \
econo

In [15]:
#import telebot
# Создаем экземпляр бота
#bot = telebot.TeleBot('5437613672:AAFNSAwxRIF5FBLZFM3i3S8kQlxKsKoZY8s')
# Функция, обрабатывающая команду /start
#@bot.message_handler(commands=["start"])
#def start(m, res=False):
    #bot.send_message(m.chat.id, 'Я на связи. Напиши мне что-нибудь )')
# Получение сообщений от юзера
#@bot.message_handler(content_types=["text"])
#def handle_text(message):
    #bot.send_message(message.chat.id, 'Вы написали: ' + message.text)
# Запускаем бота
#bot.polling(none_stop=True, interval=0)